In [1]:
import pandas as pd
import random
import json
import csv
from tqdm import tqdm

In [2]:
df = pd.read_excel('../Excel_files/Full_Poem_Dataset_12-17.xlsx')
df = df.drop(columns=['Unnamed: 0'])
records = df.to_dict('records')
random.choice(records)

{'Text': 'А как выйдет младенец биясь и крича\nтак сосед вызывает мента и врача\nи склонившись над яслями оба \nдержат тайный совет с ослецом и тельцом\nоттесняя подальше Марию с отцом \nне мелькайте мамаша я тут говорю с понятыми. \n \nИ пока ему мент указует козу \nврач глядит ему в черные эти глаза\nеврейчонка зайчонка сучонка\nи жалеет соседа бессонного пса\nи не знает какого тут сунуть упса \nсамому бы принять и поспать до второго захода. \n \nИ пока ему врач отмеряет на вес, \nмент успел обойти весь овин и завис\nибо тоже не зверь но зверяче\nозирает пеленки трусца пальтеца\nпочерневших с лица ослеца и тельца \nтут бы просто свалить но ведь сука опять дозвонится.\n \nИ пока ему каплют в развешенный рот\nон орет арамейским орет и орет \nбатя мнется Марию колбасит \nи в запаре ослец не сказал ли чего\nи как может телец утешает его \nну сказал выпивают немножко живые же люди. \n \nА орет и орет он ногами суча\nпро тельца ослеца и мента и врача\nи соседа конечно соседа\nнадрываясь и 

## Word Co-occurrence

### Initialize Spacy models

In [3]:
import spacy

In [4]:
# !python -m spacy download uk_core_news_lg

In [5]:
# !python -m spacy download ru_core_news_lg

In [6]:
nlp = spacy.load("ru_core_news_lg", disable=['attribute_ruler','ner'])

### Generate Docs

In [7]:
for rec in tqdm(records):
    recTxt = rec['Text']
    # if not isinstance(recTxt,str):
    #     recTxt = str(recTxt)
    doc = nlp(recTxt)
    rec['spacydoc'] = doc

100%|███████████████████████████████████████| 3222/3222 [02:54<00:00, 18.43it/s]


### Run cooc parser

In [8]:
import itertools
import string

In [9]:
from spacy.lang.ru import stop_words
ru_stops = stop_words.STOP_WORDS

from spacy.lang.en import stop_words
en_stops = stop_words.STOP_WORDS

from spacy.lang.uk import stop_words
uk_stops = stop_words.STOP_WORDS

from spacy.lang.es import stop_words
es_stops = stop_words.STOP_WORDS

from spacy.lang.pl import stop_words
pl_stops = stop_words.STOP_WORDS

stopwords = list(ru_stops)+list(uk_stops)
foreign_stops = list(en_stops)+list(es_stops)+list(pl_stops)
short_stops = [s for s in stopwords if len(s) < 3]
short_ru_stops = [s for s in ru_stops if len(s) < 3]

In [10]:
def isLemma(txt):
    for p in string.punctuation+'—'+'–':
        if p in txt:
            return False
    if txt.isspace():
        return False
    if txt in foreign_stops:
        return False
    # if txt in short_stops:
    #     return False
    if txt in stopwords:
        return False
    return True

In [11]:
isLemma('говорить')

False

## Before and after separate

In [21]:
for period in ['Before','After'][1:]:
    
    lens=set()
    lenCts = dict()

    networkJson = dict()
    networkJson['nodes'] = []
    networkJson['links'] = []

    curatedNodes = set()
    linkCounter = dict()
    lemmaCounter = dict()

    print('finding sufficient nodes...')

    for rec in tqdm(records):
        # if rec['Before or after'] != period:
            # continue
            
        recTxt = rec['Text']

        doc = rec['spacydoc']
        for t1 in doc:
            l1 = t1.lemma_.lower()
            lemmaCounter.setdefault(l1, 0)
            lemmaCounter[l1] += 1

    # find those nodes which occur enough times
    sufficientNodes = set()
    for lemma in lemmaCounter:
        if lemmaCounter[lemma] >= 5 and isLemma(lemma):
            sufficientNodes.add(lemma.lower())


    label2lines = dict()
    # label2recs = dict()
    label2authors = dict()

    print('parsing links...')
    for rec in tqdm(records):
        
        if rec['Before or after'] != period:
            continue
        
        doc = rec['spacydoc']
        newdoc = [t for t in doc if (t.text.isalpha() or t.text == '\n')]
        doclen = len(newdoc)
        # # line jumping
        allLines = []
        newLine = []
        for i, t in enumerate(newdoc):
            if t.text == '\n':
                allLines.append(newLine)
                newLine = []
            elif i == doclen-1:
                allLines.append(newLine)
            else:
                newLine.append(t)
        numLines = len(allLines)
        windowlength = 2

        labelFoundFromLine = dict()
        for i1 in range(numLines-windowlength):
            # all tokens in the desired range
            tokensInWindow = []
            excerptLines = []
            for line in allLines[i1:i1+windowlength]:
                excerptLines.append(' '.join([t.text for t in line]))
                for token in line:
                    tokensInWindow.append(token)
            tiw = len(tokensInWindow)
            lens.add(tiw)
            lenCts.setdefault(tiw,0)
            lenCts[tiw] += 1
            if tiw > 50:
                continue

            for ic, (t1, t2) in enumerate(itertools.combinations(tokensInWindow,2)):
                l1 = t1.lemma_.lower()
                l2 = t2.lemma_.lower()
                if l1 != l2 and l1 in sufficientNodes and l2 in sufficientNodes:
                    label = 'AND'.join(sorted([l1, l2]))

                    ### label already found in line?
                    repeatedInstance = False
                    for line in excerptLines:
                        labelFoundFromLine.setdefault((label, line), False)
                        if labelFoundFromLine[(label, line)]:
                            repeatedInstance = True
                        else:
                            labelFoundFromLine[(label, line)] = True
                    if repeatedInstance:
                        continue

                    label2authors.setdefault(label, set())
                    label2authors[label].add(rec['Author'])
#                     label2recs.setdefault(label, [])
#                     if rec not in label2recs[label]:
#                         label2recs[label].append(rec)

                    excerpt = '\n'.join(excerptLines)
                    label2lines.setdefault(label, [])
                    if excerpt not in [e['excerpt'] for e in label2lines[label]]:
                        # increase link Ct
                        linkCounter.setdefault(label, 0)
                        linkCounter[label] += 1
                        period = rec['Before or after']
                        if pd.isna(period):
                            period = ''
                        author = rec['Author']
                        if pd.isna(author):
                            author = 'Unknown'
                        label2lines[label].append({
                            'excerpt' : excerpt,
                            'author' : author,
                            'period' : period,
                            'uniqueIndex' : rec['UniqueIndex']
                        })

    node2id = dict()
    for i, node in enumerate(sufficientNodes):
        networkJson['nodes'].append({
            'id' : node,
            'totalinstances' : lemmaCounter[node]
        })
        node2id[node] = i

    for label in linkCounter:
        source = label.split('AND')[0]
        target = label.split('AND')[1]
        if lemmaCounter[source] > 9 and lemmaCounter[target] > 9:
            networkJson['links'].append({
                'source' : node2id[source],
                'target' : node2id[target],
                'sourceLemma' : source,
                'targetLemma' : target,
                'linkCt' : linkCounter[label],
                'authorCt' : list(label2authors[label])
            })
    print(sorted([l['linkCt'] for l in networkJson['links']],reverse=True)[:10])
    print(len(networkJson['links']))
    with open(f'../wordnet/data/{period}-wordnet2lines_allstops_2-21.json', 'w') as json_file:
        json.dump(networkJson, json_file, ensure_ascii = False, indent=4)
    with open(f'../wordnet/data/{period}-label2lines_2-21.json', 'w') as json_file:
        json.dump(label2lines, json_file, ensure_ascii=False, indent=4)
    print('wrote the jsons!')

finding sufficient nodes...


100%|█████████████████████████████████████| 3222/3222 [00:02<00:00, 1281.07it/s]


parsing links...


100%|██████████████████████████████████████| 3222/3222 [00:11<00:00, 291.66it/s]


[47, 33, 27, 26, 25, 24, 23, 23, 22, 22]
162859
wrote the jsons!


## Before and After Together

In [22]:
lens=set()
lenCts = dict()

networkJson = dict()
networkJson['nodes'] = []
networkJson['links'] = []

curatedNodes = set()
linkCounter = {'Before' : dict(), 'After' : dict()}
lemmaCounter = dict()

print('finding sufficient nodes...')

for rec in tqdm(records):
    # if rec['Before or after'] != period:
        # continue

    recTxt = rec['Text']

    doc = rec['spacydoc']
    for t1 in doc:
        l1 = t1.lemma_.lower()
        lemmaCounter.setdefault(l1, 0)
        lemmaCounter[l1] += 1

# find those nodes which occur enough times
sufficientNodes = set()
for lemma in lemmaCounter:
    if lemmaCounter[lemma] >= 5 and isLemma(lemma):
        sufficientNodes.add(lemma.lower())


label2lines = dict()
# label2recs = dict()
label2authors = dict()
allLabels = set()

print('parsing links...')
for rec in tqdm(records):

    period = rec['Before or after']

    doc = rec['spacydoc']
    newdoc = [t for t in doc if (t.text.isalpha() or t.text == '\n')]
    doclen = len(newdoc)
    # # line jumping
    allLines = []
    newLine = []
    for i, t in enumerate(newdoc):
        if t.text == '\n':
            allLines.append(newLine)
            newLine = []
        elif i == doclen-1:
            allLines.append(newLine)
        else:
            newLine.append(t)
    numLines = len(allLines)
    windowlength = 2

    labelFoundFromLine = dict()
    for i1 in range(numLines-windowlength):
        # all tokens in the desired range
        tokensInWindow = []
        excerptLines = []
        for line in allLines[i1:i1+windowlength]:
            excerptLines.append(' '.join([t.text for t in line]))
            for token in line:
                tokensInWindow.append(token)
        tiw = len(tokensInWindow)
        lens.add(tiw)
        lenCts.setdefault(tiw,0)
        lenCts[tiw] += 1
        if tiw > 50:
            continue

        for ic, (t1, t2) in enumerate(itertools.combinations(tokensInWindow,2)):
            l1 = t1.lemma_.lower()
            l2 = t2.lemma_.lower()
            if l1 != l2 and l1 in sufficientNodes and l2 in sufficientNodes:
                label = 'AND'.join(sorted([l1, l2]))

                ### label already found in line?
                repeatedInstance = False
                for line in excerptLines:
                    labelFoundFromLine.setdefault((label, line), False)
                    if labelFoundFromLine[(label, line)]:
                        repeatedInstance = True
                    else:
                        labelFoundFromLine[(label, line)] = True
                if repeatedInstance:
                    continue

                label2authors.setdefault(label, {'Before' : set(), 'After' : set()})
                label2authors[label][period].add(rec['Author'])


                # label2recs.setdefault(label, [])
                # if rec not in label2recs[label]:
                #     label2recs[label].append(rec)

                excerpt = '\n'.join(excerptLines)
                label2lines.setdefault(label, [])
                if excerpt not in [e['excerpt'] for e in label2lines[label]]:
                    # increase link Ct
                    for p in ['Before','After']:
                        linkCounter[p].setdefault(label, 0)
                    linkCounter[period][label] += 1
                    allLabels.add(label)

                    if pd.isna(period):
                        period = ''
                    author = rec['Author']
                    if pd.isna(author):
                        author = 'Unknown'
                    label2lines[label].append({
                        'excerpt' : excerpt,
                        'author' : author,
                        'period' : period,
                        'uniqueIndex' : rec['UniqueIndex']
                    })

node2id = dict()
for i, node in enumerate(sufficientNodes):
    networkJson['nodes'].append({
        'id' : node,
        'totalinstances' : lemmaCounter[node]
    })
    node2id[node] = i

for label in allLabels:
    source = label.split('AND')[0]
    target = label.split('AND')[1]
    if lemmaCounter[source] > 9 and lemmaCounter[target] > 9:
        label2authors[label]['Before'] = list(label2authors[label]['Before'])
        label2authors[label]['After'] = list(label2authors[label]['After'])
        
        networkJson['links'].append({
            'source' : node2id[source],
            'target' : node2id[target],
            'sourceLemma' : source,
            'targetLemma' : target,
            'linkCtBefore' : linkCounter['Before'][label],
            'linkCtAfter' : linkCounter['After'][label],
            'authors' : label2authors[label],
        })
print(sorted([l['linkCtBefore'] for l in networkJson['links']],reverse=True)[:10])
print(len(networkJson['links']))
with open(f'../matrices/allLinkData3-3_2.json', 'w') as json_file:
    json.dump(networkJson, json_file, ensure_ascii = False, indent=4)

print('wrote the json!')

finding sufficient nodes...


100%|██████████████████████████████████████| 3222/3222 [00:17<00:00, 181.42it/s]


parsing links...


100%|██████████████████████████████████████| 3222/3222 [00:20<00:00, 156.51it/s]


[37, 30, 26, 23, 21, 19, 19, 18, 18, 17]
310725
wrote the json!


In [15]:
label2authors.keys()

dict_keys(['баварияANDзима', 'зимаANDпохожий', 'другуюANDзима', 'баварияANDпохожий', 'баварияANDдругую', 'другуюANDпохожий', 'блуждатьANDдругую', 'другуюANDодиночка', 'блуждатьANDзима', 'зимаANDодиночка', 'блуждатьANDодиночка', 'блуждатьANDискать', 'блуждатьANDрифма', 'искатьANDодиночка', 'одиночкаANDрифма', 'искатьANDрифма', 'искатьANDслабый', 'вестьANDискать', 'рифмаANDслабый', 'вестьANDрифма', 'вестьANDслабый', 'девицаANDслабый', 'пределANDслабый', 'вестьANDдевица', 'вестьANDпредел', 'девицаANDпредел', 'девицаANDпойти', 'пойтиANDпредел', 'зимаANDпойти', 'баварияANDпойти', 'известныйANDпойти', 'зимаANDизвестный', 'баварияANDизвестный', 'затягиватьANDзима', 'баварияANDзатягивать', 'затягиватьANDизвестный', 'затягиватьANDмелодия', 'дебилANDзатягивать', 'дебилANDмелодия', 'мелодияANDтерзать', 'душаANDмелодия', 'дебилANDтерзать', 'дебилANDдуша', 'душаANDтерзать', 'терзатьANDходить', 'паркANDтерзать', 'сокANDтерзать', 'душаANDходить', 'душаANDпарк', 'душаANDсок', 'паркANDходить', 'сокANDх

In [36]:
label2lines[a['sourceLemma']+'AND'+a['targetLemma']]

[{'excerpt': 'Видимо когда что то привычное\nменяет свою форму',
  'author': 'Виктор Плавский',
  'period': 'After',
  'uniqueIndex': 172},
 {'excerpt': 'неразрывного одного объема\nчуть меняющего форму когда в него бьют',
  'author': 'Василий Бородин',
  'period': 'Before',
  'uniqueIndex': 2896}]